In [ ]:
# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Training utils
# import utils_xgboost

# Optuna
# import optuna

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# # Modelos
# from sklearn.linear_model import LinearRegression # for building a linear regression model
# from sklearn.svm import SVR # for building SVR model
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.multioutput import MultiOutputRegressor
# import xgboost as xgb
# from sklearn.model_selection import train_test_split

# Metricas
# from sklearn.metrics import mean_absolute_error #MAE
# from sklearn.metrics import mean_absolute_percentage_error #MAPE
# from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

In [ ]:
datos = pd.read_csv('datos/230109_con_MP_DIA_ANTERIOR.csv', parse_dates = ['FECHAHORA'])

In [ ]:
datos.info()

# Paso 1: Eliminamos datos MP_ANTERIOR y agregamos PRONOSTICOS CLIMATICOS

In [ ]:
estaciones = []
for i in range(1, 11):
    estaciones.append(datos[datos['ESTACION'] == i])
    
estaciones[9]

In [ ]:

    
df_lista = []

for i in range(0, 10):
    df_i = estaciones[i].copy()
    df_i = df_i.sort_values(by = ['FECHAHORA'])
    df_i = df_i.reset_index(drop = True)
#     num_filas = df_i.shape[0]
    
    # Identifico el indice de la ultima medicion y la de un dia despues
    last_index = df_i.index[-1]
    day_before_index = df_i.index[-289]
    
    # elimino el ultimo dia de mediciones 
    df_final = df_i.drop(list(range(last_index, day_before_index, -1)))
    df_final = df_final.reset_index(drop = True)
    
    # creo un df auxiliar con solo las mediciones de temp, humedad y presion y elimino las primeras 288 filas
    
    df_i_thp = df_i[['TEMPERATURA', 'HUMEDAD', 'PRESION']].drop(list(range(0, 288)))

    df_i_thp = df_i_thp.reset_index(drop = True)

    df_final[['TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO']] = df_i_thp
    
    df_lista.append(df_final)
    
df = pd.concat(df_lista)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
e = 3

e_df = df[df['ESTACION'] == e]

fig.add_trace(
    go.Scatter( x = list(e_df.FECHAHORA), y = list(e_df.HUMEDAD), name = 'HUMEDAD'))
fig.add_trace(
    go.Scatter( x = list(e_df.FECHAHORA), y = list(e_df.HUMEDAD_PRONOSTICO), name = 'PRONOSTICO'))

fig.update_layout( title_text = "HUMEDAD VS PRONOSTICO")

In [ ]:
# checkeamos que tenga todo lo que tiene que tener

df.info()

In [ ]:
# Guardamos el dataframe completito

df.to_csv('datos/230127_CON_PRONOSTICOS_THP.csv', index = False)

# Paso 2: Seleccionamos solo datos de los primeros 18 meses de mediciones

## Analizamos primero los datos de las estaciones 7 y 10 que tenian faltantes

In [ ]:
fig = go.Figure()

e = 7

plot_df_val = df[df['ESTACION'] == e]

fig.add_trace(
    go.Scatter( x = list(plot_df_val.FECHAHORA), y = list(plot_df_val.AQI_MP2_5), name = 'AQI'))

fig.add_trace(
    go.Scatter( x = list(plot_df_val.FECHAHORA), y = list(plot_df_val.MP2_5), name = 'MP2.5'))

fig.update_layout( title_text = "Estacion "+str(e))

Estacion 7: Hasta 30 de octubre 2020. Desde 8 de abril 2019

Estacion 10: Hasta 22 de octubre. Desde 31 de marzo 2019

## Vemos que justo alcanza los 18 meses. Entonces hacemos una BD con solo los datos de los primeros 18 meses de cada estacion.

In [ ]:
estaciones = []
for i in range(1, 11):
    estaciones = range(1, 11)
    
df_lista = []
    
for i in estaciones:
    print('estacion ' + str(i))
    df_i = df[df['ESTACION'] == i].copy()
    fecha_1 = df_i.FECHAHORA.min()
    print(fecha_1)
    delta = relativedelta(months = 18)
    fecha_n = fecha_1 + delta
    print(fecha_n)
    print('\n')
    
    df_i = df_i[df_i['FECHAHORA'] <= fecha_n]
    df_lista.append(df_i)
    
df_18 = pd.concat(df_lista)

df_18.head()

In [ ]:
df_18.info()

## Separamos en df_train y df_test

df_train = 15 meses


df_test = 3 meses

In [ ]:
delta_train = relativedelta(months = 15)

df_train_list = []
df_test_list = []

for i in estaciones:
    print('estacion ' + str(i))
    df_i = df_18[df_18['ESTACION'] == i].copy()
    
    fecha_1_i = df_i.FECHAHORA.min()
    print(fecha_1_i)
    
    fecha_n_i = fecha_1_i + delta_train
    print(fecha_n_i)
    print('\n')
    
    df_train_i = df_i[df_i['FECHAHORA'] <= fecha_n_i]
    df_test_i = df_i[df_i['FECHAHORA'] > fecha_n_i]
    
    df_train_list.append(df_train_i)
    df_test_list.append(df_test_i)

df_train = pd.concat(df_train_list)
df_train = df_train.reset_index(drop = True)
df_test = pd.concat(df_test_list)
df_test = df_test.reset_index(drop = True)

In [ ]:
# checkeamos df_train y df_test

df_test.info()

In [ ]:
df_train.to_csv('datos/230127_train_ESTACIONES.csv', index = False)
df_test.to_csv('datos/230127_test_ESTACIONES.csv', index = False)

In [ ]:
# Ultimo check 

train = pd.read_csv('datos/230127_test_ESTACIONES.csv', parse_dates = ['FECHAHORA'])

train.head()